# Non-Normal Distribution

![robot.png](https://live.staticflickr.com/65535/54430891765_38ef5cb61e_z.jpg)

*Image generated using ChatGPT.*

## Introduction

Noise has accompanied us for at least as long as we've been recording observations of any kind. Is this because we don’t live in a world of classical philosophical abstractions, or is the truth far more mundane—into the camera frame, telescope lens, snippet of text, or sound recording, entirely unwanted signals often enter, which—indeed—co-create reality, but at the moment of observation, we would prefer to avoid them. In the context of this task, such surplus information superimposed on the base (true) information will be referred to as **noise**.

Noise is studied and described mathematically—in the exact sciences, particularly in *information theory*. In computer graphics, we define noise (a *noise function*) as:

$$f: X → X$$

where X is a defined domain of images. For grayscale images of size 28x28 encoded in the range of real numbers [0, 1]:

$$X = [0,1]^{28 * 28} $$

A reasonable assumption is that f significantly differs from the identity function, i.e., it significantly distorts the base image.

*Gaussian noise* is defined based on the **Gaussian distribution**, whose probability density is given by the formula:

$$f_{\mu, \sigma}(x) = \frac{1}{\sigma \sqrt{2π}}e^{\frac{-(x - \mu)^2}{2\sigma^2}}$$

**The Gaussian distribution** is parameterized by two constants: $\mu$ – *mean* and $\sigma$ – *standard deviation*, or equivalently: $\mu$ – *mean* and $\sigma^2$ – *variance* (the square of the standard deviation). Popular computation libraries include implementations for *sampling* from this distribution. To add noise to an image parameterized by such a distribution, we sample from it an array equal in size to the image, add the noise to the image (add *pixel-wise*), and then ensure that the pixel values remain in the [0, 1] range (a **clamp** function).

```py
######################### DO NOT MODIFY THIS CELL ##########################

def add_normal_noise(image, mean=0, std=0.2):
    """Adds normal (Gaussian) noise to the image."""
    noise = torch.distributions.normal.Normal(mean, std)
    noise = noise.sample(image.size())
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0.0, 1.0)
```


$$    
  f(x) =
  \begin{cases}
    \frac{1}{b-a}, & \text{for } a \leq x \leq b \\
    0
  \end{cases}
\
$$

When adding uniform noise to an image, we proceed analogously to Gaussian noise. We draw a sample from the distribution, add it to the image, and any pixels going outside the $[0,1]$ range are clamped to the corresponding boundary.

```py
######################### DO NOT MODIFY THIS CELL ##########################

def add_uniform_noise(image, low=-0.5, high=0.5):
    """Adds uniform noise to the image."""
    noise = torch.empty(image.size()).uniform_(low, high)
    noisy_image = image + noise
    return torch.clamp(noisy_image, 0.0, 1.0)
```

## Task

Imagine you are an image processing specialist working at a company focused on image analysis and reconstruction. Your team is developing a system that can not only remove noise from images but also identify its type and parameters, which can provide valuable information about the source of disturbances.

Your task is to design and train a single neural network architecture capable of simultaneously accomplishing three objectives:

* **Image denoising** — restoring the original appearance of images corrupted by one of two types of noise: Gaussian or uniform;
* **Noise type classification** — determining whether the image was corrupted by Gaussian noise (label 0) or uniform noise (label 1);
* **Estimation of Gaussian noise parameters** — for images corrupted by Gaussian noise, the model should additionally estimate the parameters of this noise: mean \$\mu\$ and standard deviation \$\sigma\$.

**Note that each individual image is corrupted by a randomly chosen distribution with randomly chosen (potentially different across the entire dataset) parameters.**

### Data

You have access to the following data for this task:

* **Training dataset**, containing both original images and their noised versions along with noise type labels;
* **Validation dataset**, which will help you evaluate the quality of your model during training.

We have prepared a dataloader for you. In the training dataset, each example consists of:

* Image before noising - key `['original']`
* Image after noising - key `['noised']`
* Noise type label - key `['label']`
* Noise parameters - key `['params']` (available only in validation and test sets)

The following illustration shows an example process of generating noised images for both noise types, parameterized by example arguments.

![noise\_schema.png](https://live.staticflickr.com/65535/54429663172_1014ff20d7_z.jpg")

Your solution will be finally tested on the Competition Platform on a hidden test set balanced with respect to noise types, and the images there will have the same characteristics as those provided to participants.

### Evaluation Criteria

As you might expect, the evaluation will assess four key aspects of your solution:

1. **Binary noise classification accuracy** (weight 25%) — how effectively the model recognizes the noise type:

$$
  accuracyScore =
  \begin{cases}
    \frac{accuracy - 0.5}{0.45}, & \text{for } 0.5 < accuracy < 0.95 \\
    0.0, & \text{for } accuracy \leq 0.5 \\
    1.0, & \text{for } 0.95 \leq accuracy
  \end{cases}
\
$$

2. **Image reconstruction quality** (weight 25%) — measured by PSNR (Peak Signal-to-Noise Ratio) metric:

$$
  psnrScore =
  \begin{cases}
    \frac{PSNR - 10}{6}, & \text{for } 10 < PSNR < 16 \\
    0.0, & \text{for } PSNR \leq 10 \\
    1.0, & \text{for } 16 \leq PSNR
  \end{cases}
\
$$

where PSNR is defined as:

$PSNR = 10 \log_{10}\frac{MAX_{I}^{2}}{MSE},$

where \$MAX\_{I}\$ is the maximum possible pixel value for the given representation, and in our case \$MAX\_{I} = 1\$.

3. **Accuracy of the Gaussian noise mean \$\mu\$ estimation** (weight 25%) — measured by mean squared error (MSE) calculated only over test samples with label 0 (Gaussian noise):

$$
  meanMseScore =
  \begin{cases}
   1.0, & \text{for } MSE < 0.005 \\
   0.0, & \text{otherwise}
  \end{cases}
\
$$

4. **Accuracy of the Gaussian noise standard deviation \$\sigma\$ estimation** (weight 25%) — also measured by MSE and calculated over test samples with label 0 (Gaussian noise):

$$
  stdMseScore =
  \begin{cases}
   1.0, & \text{for } MSE < 0.005 \\
   0.0, & \text{otherwise}
  \end{cases}
\
$$

**Final Scoring Formula**

The final score is the weighted sum of the above metrics according to the formula:
$finalScore = 25 \cdot accuracyScore + 25 \cdot psnrScore + 25 \cdot meanMseScore + 25 \cdot stdMseScore$

For this task you can score between 0 and 100 points, where:

* Values close to 0 indicate a weak solution;
* Values close to 100 indicate an excellent solution that effectively classifies the noise type, reconstructs original images, and precisely estimates Gaussian noise parameters.

## Constraints

* You may use only the training dataset for model training.
* Your solution will be tested on the Competition Platform without internet access and in an environment with GPU.
* Evaluation of your final solution on the Competition Platform must not exceed 5 minutes with GPU.

## Submission Files

This notebook supplemented with your solution (see class `Model`).


# Starter Code
In this section, we initialize the environment by importing the necessary libraries and functions. The prepared code will facilitate your efficient handling of data and building the proper solution.

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

FINAL_EVALUATION_MODE = False  # During the evaluation of your solution, we will set this value to True

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

import os
import pickle
import random
from typing import Dict, List, Tuple
from collections.abc import Callable
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

assert torch.cuda.is_available(), "CUDA niedostępna!"

print("Device:", DEVICE)

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

seed = 42

os.environ["PYTHONHASHSEED"] = str(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################
# Cell containing helper functions for visualizing results

def plot_samples(dataset: Dataset, num_images: int = 6, title: str = "") -> None:
    """
    Function to display examples of original and noised images

    Args:
        dataset (Dataset): Dataset.
        num_images (int): Number of images to display.
    """
    fig, axs = plt.subplots(2, num_images, figsize=(2 * num_images, 4))
    fig.suptitle(title, fontsize=10)

    for i in range(num_images):
        sample = dataset[i]
        original_image = sample["original"]
        noised_image = sample["noised"]
        label = sample["label"]
        params = sample.get("params", None)

        column_title = f"Example {i+1}\nLabel: {label.item():.0f}"

        if params is not None:
            if label.item() == 0:
                column_title += f"\nμ: {params[0].item():.2f}\nσ: {params[1].item():.2f}"
            else:
                column_title += f"\nlow: {params[0].item():.2f}\nhigh: {params[1].item():.2f}"
        else:
            if label.item() == 0:
                column_title += "\nμ: None\nσ: None"
            else:
                column_title += "\nlow: None\nhigh: None"

        column_title += "\n\nOriginal"

        axs[0, i].set_title(column_title, fontsize=8, pad=5)
        axs[0, i].imshow(original_image.squeeze(), cmap="gray")
        axs[0, i].axis("off")

        axs[1, i].set_title("Noised", fontsize=8, pad=5)
        axs[1, i].imshow(noised_image.squeeze(), cmap="gray")
        axs[1, i].axis("off")

    plt.tight_layout()
    plt.show()


def plot_results(model: nn.Module, examples: Dict, num_images: int = 6) -> None:
    """
    Function to display examples of noised images and images denoised by the model

    Args:
        model (nn.Module): Model for image denoising.
        examples (dict): Dictionary containing image examples.
        num_images (int): Number of images to display.
    """
    model.eval()
    noisy_images = examples["noised"][:num_images].to(DEVICE)
    clean_images = examples["original"][:num_images]
    label = examples["label"][:num_images]
    params = examples["params"][:num_images]
    mean_real = params[:, 0].view(-1, 1)
    std_real = params[:, 1].view(-1, 1)

    with torch.no_grad():
        output_images, predictions, mean_pred, std_pred = model(noisy_images.to(DEVICE))

    fig, axs = plt.subplots(3, num_images, figsize=(2 * num_images, 6))

    for i in range(num_images):
        column_title = (
            f"Example {i+1}\n"
            f"Label: {float(predictions[i].item() > 0.5):.0f}/{label[i].item():.0f}\n"
        )

        if label[i].item() == 0:
            column_title += (
                f"μ: {mean_pred[i].item():.2f}/{mean_real[i].item():.2f}\n"
                f"σ: {std_pred[i].item():.2f}/{std_real[i].item():.2f}\n"
            )
        else:
            column_title += "\n\n"

        column_title += "\nNoised"

        axs[0, i].set_title(column_title, fontsize=8, pad=5)
        axs[0, i].imshow(noisy_images[i].cpu().squeeze(), cmap="gray")
        axs[0, i].axis("off")

        axs[1, i].set_title("Denoised", fontsize=8, pad=5)
        axs[1, i].imshow(output_images[i].cpu().squeeze(), cmap="gray")
        axs[1, i].axis("off")

        axs[2, i].set_title("Original", fontsize=8, pad=5)
        axs[2, i].imshow(clean_images[i].squeeze(), cmap="gray")
        axs[2, i].axis("off")

    fig.text(0.5, 0.01, "Format: Prediction/Label", ha='center', fontsize=10)

    plt.tight_layout()
    plt.subplots_adjust(top=0.85, bottom=0.1)
    plt.show()

## Data Loading
The following code will load and properly prepare the data.

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################
# Cell containing helper functions for data preparation.

class NoisedDataset(Dataset):
    """
    Dataset loaded from a pickle file.

    Args:
        file_path (str): Path to the pickle file containing data.
        transform (callable, optional): Transformations applied to images and labels.
    """
    def __init__(self, pickle_file, transform=None):
        self.transform = transform

        with open(pickle_file, 'rb') as f:
            self.data = pickle.load(f)

        self.has_params = 'params' in self.data[0]

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        sample = self.data[idx]
        original_image = sample['original']
        noised_image = sample['noised']
        label = sample['label']

        if self.has_params:
            data = {'params': sample['params']}
        else:
            data = {}

        if self.transform:
            original_image = self.transform(original_image)
            noised_image = self.transform(noised_image)

        data.update({
            'original': original_image,
            'noised': noised_image,
            'label': label
        })

        return data


def setup_data(
        train_transform: Callable | None = None,
        val_transform: Callable | None = None,
        root: str = './'
    ) -> Tuple[Dataset, Dataset]:
    """
    Prepares training and validation datasets, downloading them if necessary.

    Args:
        train_transform (callable, optional): Augmentations for training set.
        val_transform (callable, optional): Augmentations for validation set.
        root (str, optional): Base directory for data files.

    Returns:
        tuple: Datasets (train_ds, val_ds).
    """
    if train_transform is None:
        train_transform = transforms.Compose([transforms.ToTensor()])
    if val_transform is None:
        val_transform = transforms.Compose([transforms.ToTensor()])

    train_file = root+'train.pkl'
    val_file = root+'val.pkl'

    if not os.path.exists(root):
        os.makedirs(root)

    train_ds = NoisedDataset(train_file, transform=train_transform)
    val_ds = NoisedDataset(val_file, transform=val_transform)

    return train_ds, val_ds

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

train_ds, val_ds = setup_data(root="./")

if not FINAL_EVALUATION_MODE:
    print("Number of images in training set:", len(train_ds), ", number of images in validation set:", len(val_ds))
    plot_samples(train_ds, num_images=6, title="Training set")
    plot_samples(val_ds, num_images=6, title="Validation set")

## Evaluation Code
Code similar to the one below will be used to evaluate your solution on the test set.

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################
# Cell containing helper functions to compute model metrics

def compute_psnr(input_image: torch.Tensor, target_image: torch.Tensor) -> torch.Tensor:
    """
    Function to calculate PSNR between two images.

    Args:
        input_image (torch.Tensor): First image.
        target_image (torch.Tensor): Second image.

    Returns:
        torch.Tensor: PSNR value.
    """
    mse = F.mse_loss(input_image, target_image)
    if mse == 0:
        return 100
    return 10 * torch.log10(1 / mse)

def model_eval(model: nn.Module, dataloader: DataLoader, device: str = DEVICE) -> Tuple[float, float, float, float]:
    """
    Function to evaluate the model on a dataset.

    Args:
        model (nn.Module): Model to evaluate.
        dataloader (DataLoader): DataLoader with evaluation data.
        device (str, optional): Device on which to perform evaluation.

    Returns:
        tuple: Tuple containing metric values (PSNR, accuracy, MSE for parameter 1, MSE for parameter 2).
    """
    model.eval()
    model.to(device)

    # Initialize variables to store results
    psnr = 0
    correct = 0
    mean_mse = 0
    std_mse = 0

    total_samples = 0
    total_label0_samples = 0

    with torch.no_grad():
        for data in dataloader:
            noised_images = data["noised"].to(device)
            original_images = data["original"].to(device)
            labels = data["label"].to(device)
            params = data["params"].to(device)
            batch_size = len(labels)

            mean_real = params[:, 0].view(-1, 1)
            std_real = params[:, 1].view(-1, 1)

            output_images, labels_pred, mean_pred, std_pred = model(noised_images)

            # Calculate classification accuracy
            correct += ((labels_pred >= 0.5).float().view(-1) == labels).sum().item()

            # Calculate PSNR
            psnr += compute_psnr(output_images, original_images) * batch_size

            # Calculate MSE for parameters when label equals 0
            label0_mask = (labels == 0)
            num_label0 = label0_mask.sum().item()

            if num_label0 > 0:
                mean_mse += F.mse_loss(mean_pred[label0_mask], mean_real[label0_mask], reduction='sum')
                std_mse += F.mse_loss(std_pred[label0_mask], std_real[label0_mask], reduction='sum')

            total_samples += batch_size
            total_label0_samples += num_label0

    # Calculate average metric values
    psnr /= total_samples
    accuracy = correct / total_samples
    mean_mse /= total_label0_samples
    std_mse /= total_label0_samples

    return psnr.item(), accuracy, mean_mse.item(), std_mse.item()


In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################
# Cell containing helper functions to score your solution

def calculate_score(
    psnr: float, accuracy: float, mean_mse: float, std_mse: float
) -> Tuple[float, float, float, float]:
    """
    Function to calculate the task points based on model metrics.

    Args:
        psnr (float): PSNR value.
        accuracy (float): Classification accuracy.
        mean_mse (float): MSE for parameter 1.
        std_mse (float): MSE for parameter 2.

    Returns:
        tuple: Tuple containing task points (PSNR, accuracy, MSE for parameter 1, MSE for parameter 2).
    """

    def scale(x, lower=0.0, upper=1.0, max_points=1.0):
        scaled = min(max(x, lower), upper)
        return (scaled - lower) / (upper - lower) * max_points

    accuracy_score = scale(accuracy, lower=0.5, upper=0.95)
    psnr_score = scale(psnr, lower=10.0, upper=16.0)

    mean_score = 0.0
    if mean_mse < 0.005:
        mean_score = 1.0

    std_score = 0.0
    if std_mse < 0.005:
        std_score = 1.0

    return psnr_score, accuracy_score, mean_score, std_score


def grade_solution(model: nn.Module, dataloader: DataLoader) -> float:
    """
    Function to grade the model on the validation dataset.

    Args:
        model (nn.Module): Model to evaluate.
        dataloader (DataLoader): DataLoader with evaluation data.

    Returns:
        float: Number of points awarded for the task.
    """
    psnr, accuracy, mean_mse, std_mse = model_eval(model, dataloader)
    psnr_score, accuracy_score, mean_score, std_score = calculate_score(
        psnr, accuracy, mean_mse, std_mse
    )
    score = round(
        psnr_score * 25 + accuracy_score * 25 + mean_score * 25 + std_score * 25
    )

    # Round to integer, range [0, 100]
    score = round(score)

    print(
        f"Metrics on validation set\n"
        f"psnr: {psnr:.2f}, accuracy: {accuracy:.2f}, mean_mse: {mean_mse:.6f}, std_mse: {std_mse:.6f}\n"
    )
    print(
        f"Partial points for the task\n"
        f"psnr: {(psnr_score * 25):.2f}, accuracy: {(accuracy_score * 25):.2f}, mean_mse: {(mean_score * 25):.2f}, std_mse: {(std_score * 25):.2f}\n"
    )
    print(f"Estimated total points for the task: {score}")
    return score

## Your Solution
You should place your solution in this section only. Make changes exclusively here!

In [ ]:
# definitions of augmentations for training and validation datasets. None by default means no augmentations
train_transform = None
val_transform = None

# batch size
BATCH_SIZE: int = 64

train_ds, val_ds = setup_data(train_transform, val_transform, root="./")

In [ ]:
# create and train your model here

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        """
        Transformation performed by the model.

        Args:
            x (torch.Tensor): Input image [B, 1, H, W].

        Returns:
            Returned result as a tuple:
            torch.Tensor: Output image [B, 1, H, W].
            torch.Tensor: Classification predictions [B, 1].
            torch.Tensor: Parameter 1 [B, 1].
            torch.Tensor: Parameter 2 [B, 1].
        """
        device = x.device
        return (
            torch.rand_like(x, device=device),
            torch.rand(x.shape[0], 1, device=device),
            torch.randn(x.shape[0], 1, device=device),
            torch.randn(x.shape[0], 1, device=device)
        )


def train_model() -> Model:
    """Create and train the model"""
    return Model().to(DEVICE)


your_model = train_model()

## Evaluation
Running the cell below will allow you to check how many points your solution would score on the validation data. Before submitting, make sure the entire notebook runs from start to finish without errors and without any user intervention after choosing "Run All".

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

val_dataloader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

if not FINAL_EVALUATION_MODE:
    grade_solution(your_model, val_dataloader)
    examples = next(iter(val_dataloader))
    plot_results(your_model, examples, num_images=6)

During evaluation, the model will be saved as `your_model.pkl` and assessed on the test set.

In [ ]:
######################### DO NOT MODIFY THIS CELL ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    # Gdy model posiada parametry, ustaw go w trybie ewaluacji i przenieś na CPU
    if list(your_model.parameters()):
        your_model.eval()
        your_model.cpu()

    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(your_model, f)